In [1]:
%matplotlib inline
import scipy as sc
import pandas as pd
from sklearn import ensemble, linear_model, feature_selection, feature_extraction, neural_network, preprocessing, model_selection, pipeline
import tensorflow as tf
from tensorflow.contrib import learn as ln 
import xgboost

/home/ide/.virtualenvs/data/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv('train.csv')

In [3]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
test = pd.read_csv('test.csv')

In [5]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [6]:
X=train.iloc[:,1:-1]
r=train.iloc[:,-1]
#X=X.T.dropna().T

In [7]:
Xtt=test.iloc[:,1:]
#Xtt=Xtt.T.dropna().T

In [8]:
allx = pd.concat([X,Xtt])
allx=pd.get_dummies(allx)
X=allx.iloc[:X.shape[0]]
Xtt=allx.iloc[X.shape[0]:]

X,Xt,r,rt=model_selection.train_test_split(X,r,test_size=.2)

X.fillna(0, inplace=True)
Xt.fillna(0, inplace=True)
Xtt.fillna(0, inplace=True)

mod = pipeline.make_pipeline(
    feature_extraction.DictVectorizer(sparse=False),
    preprocessing.StandardScaler(),
    linear_model.BayesianRidge(),
)
mod.fit(X,r)
mod.score(Xt,rt)

mod = pipeline.make_pipeline(
    ensemble.ExtraTreesRegressor(n_estimators=1000),
)
mod.fit(X,r)
mod.score(Xt,rt)

mod = pipeline.make_pipeline(
    ensemble.RandomForestRegressor(n_estimators=1000),
)
mod.fit(X,r)
mod.score(Xt,rt)

mod=xgboost.XGBRegressor(n_estimators=1000, reg_lambda=.1)
mod.fit(X,r)
mod.score(Xt,rt)

In [12]:
rscv=model_selection.RandomizedSearchCV(
    xgboost.XGBRegressor(n_estimators=1000, reg_lambda=.1),
    dict(n_estimators=sc.stats.randint(100,1000), reg_lambda=sc.stats.expon(scale=.1)),
    cv=10,n_iter=30,verbose=9999,
)
rscv.fit(X,r)
#print(rscv.score(Xt,rt))
print(rscv.best_params_)

Fitting 10 folds for each of 30 candidates, totalling 300 fits
[CV] n_estimators=987, reg_lambda=0.277012845523 .....................
[CV]  n_estimators=987, reg_lambda=0.277012845523, score=0.886438, total=   4.2s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.3s remaining:    0.0s
[CV] n_estimators=987, reg_lambda=0.277012845523 .....................
[CV]  n_estimators=987, reg_lambda=0.277012845523, score=0.858723, total=   3.4s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    7.7s remaining:    0.0s
[CV] n_estimators=987, reg_lambda=0.277012845523 .....................
[CV]  n_estimators=987, reg_lambda=0.277012845523, score=0.941071, total=   3.1s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   10.8s remaining:    0.0s
[CV] n_estimators=987, reg_lambda=0.277012845523 .....................
[CV]  n_estimators=987, reg_lambda=0.277012845523, score=0.901057, total=   3.2s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   14.0s remaining:    0.0s
[CV] n_es

mod=rscv.best_estimator_
mod.fit(pd.concat([X,Xt]), pd.concat([r,rt]))

In [ ]:
mod=rscv

In [14]:
mod.predict(Xtt)

array([ 119947.46875 ,  160910.125   ,  186513.734375, ...,  148292.84375 ,
        118315.53125 ,  226195.265625], dtype=float32)

mod = pipeline.make_pipeline(
    feature_extraction.DictVectorizer(sparse=False),
    preprocessing.StandardScaler(),
    neural_network.MLPRegressor(hidden_layer_sizes=[20,], max_iter=10000, learning_rate_init=.01, activation='logistic'),
)
mod.fit(X,r)
mod.score(Xt,rt)

mod = pipeline.make_pipeline(
    feature_extraction.DictVectorizer(sparse=False),
    preprocessing.StandardScaler(),
)
xm = mod.fit_transform(X)
xtm = mod.transform(Xt)
fc = ln.infer_real_valued_columns_from_input(xm)
clf = ln.DNNRegressor(
    hidden_units=[100,100,100,100],
    feature_columns=fc,
    model_dir="/tmp/model3",
    config=tf.contrib.learn.RunConfig(save_checkpoints_secs=1))
valmon = ln.monitors.ValidationMonitor(
    xtm,
    rt,
    every_n_steps=50,
    early_stopping_metric="loss",
    early_stopping_metric_minimize=True,
    early_stopping_rounds=100)

clf.fit(xm,r,monitors=[valmon])

u=clf.evaluate(xtm,rt)['loss']
v=sum((r-r.mean())**2)
(1-u/v)

mod = pipeline.make_pipeline(
    ensemble.GradientBoostingRegressor(n_estimators=1000),
)
mod.fit(X,r)
mod.score(Xt,rt)

mod.predict(Xtt)

In [15]:
res = pd.DataFrame(columns=['Id','SalePrice'])
res.Id = test.Id

In [16]:
res.SalePrice=mod.predict(Xtt)
#res.SalePrice=clf.predict(mod.transform(Xtt))

In [17]:
res.to_csv('subm.csv', index=None)